In [2]:
import os
import ray
import ray.util
from ray.util.client import ray as rayclient

In [3]:
headhost = os.environ['RAY_CLUSTER']

if not rayclient.is_connected():
    ray.util.connect('{ray_head}:10001'.format(ray_head=headhost))

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

In [5]:

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(
        num_actors=2,  # Number of remote actors
        cpus_per_actor=1))

bst.save_model("model.xgb")


(pid=288) 2021-05-25 22:52:12,077	INFO main.py:853 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(pid=288) 2021-05-25 22:52:13,787	INFO main.py:898 -- [RayXGBoost] Starting XGBoost training.
(pid=370) [22:52:13] task [xgboost.ray]:139992593665552 got new rank 0
(pid=120, ip=10.217.0.242) [22:52:14] task [xgboost.ray]:140317590049400 got new rank 1
(pid=288) 2021-05-25 22:52:15,022	INFO main.py:1358 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 57.10 seconds (1.23 pure XGBoost training time).


In [6]:
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

Final training error: 0.0053


The node with node id: 460b1e2d7f75d2fe5b409411c19267759f10f6d02f1c467eb14b9239 and ip: 10.217.0.242 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
